In [1]:
# pip install langchain langchain_community python-dotenv

In [2]:
# Langsmith tracing
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("langsmith_api")

In [3]:
from langchain_community.chat_models import ChatOllama

local_llm = "llama3"
model = ChatOllama(model=local_llm, format="json", temperature=0)
# Utilize format="json" quando a saída esperada do LLM é um JSON.

from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    Você é um assistente de identificação e tradução de idiomas.
    Identifique qual é o idioma da frase de entrada do usuário.
    Depois disso deve traduzir para o idioma Italiano.
    Provenha um JSON com duas chaves, sendo elas 'idioma' com o idioma identificado escrito em português e em lowercase, e 'tradução' com a tradução para o Italiano.
    Retorne somente o JSON sem nenhuma explicação adicional.
     <|eot_id|><|start_header_id|>user<|end_header_id|>
    Aqui está a frase do usuário: {sentence} <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["sentence"],
)

In [4]:
from langchain_core.output_parsers import JsonOutputParser
# Use JsonOutputParser quando a saída esperada do LLM é um JSON.

chain = prompt | model | JsonOutputParser()

sentence = "Eu moro em uma casa com o muro vermelho"
answer = chain.invoke({"sentence": sentence})
print(answer)

sentence = "I live in New York and I love it"
answer = chain.invoke({"sentence": sentence})
print(answer)

sentence = "日本語を話さない"
answer = chain.invoke({"sentence": sentence})
print(answer)

sentence = "¿Cuál es el nombre de su padre?"
answer = chain.invoke({"sentence": sentence})
print(answer)

{'idioma': 'português', 'tradução': 'Io abito in una casa con il muro rosso'}
{'idioma': 'inglês', 'tradução': "Io vivo a New York e l'amo"}
{'idioma': 'japonês', 'tradução': 'Non parlo giapponese'}
{'idioma': 'espanhol', 'tradução': 'Qual è il nome del suo padre?'}


In [7]:
from langchain_core.output_parsers import StrOutputParser
# Use StrOutputParser quando a saída esperada do LLM for um texto corrido.

model = ChatOllama(model=local_llm, temperature=0.8)

prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    Você é um assistente de tradução de idiomas e contador de histórias.
    Traduza a frase para o idioma Italiano e depois continue contando uma pequena história dando sequência à frase.
    Retorne somente a frase traduzida junto com o texto da história em Italiano em um mesmo parágrafo sem quebras de linha e sem nenhuma explicação adicional.
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Aqui está a frase do usuário: {sentence} 
    História: \n <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["sentence"],
)

In [8]:
chain = prompt | model | StrOutputParser()

sentence = "Eu moro em uma casa com o muro vermelho"
answer = chain.invoke({"sentence": sentence})
print(answer)

sentence = "I live in New York and I love it"
answer = chain.invoke({"sentence": sentence})
print(answer)

sentence = "日本語を話さない"
answer = chain.invoke({"sentence": sentence})
print(answer)

sentence = "¿Cuál es el nombre de su padre?"
answer = chain.invoke({"sentence": sentence})
print(answer)

Io abito in una casa con un muro rosso. La casa si trova in mezzo al giardino di mia nonna, che è un luogo magico dove crescono fiori e arbusti che sembrano avere vita propria. Il mio migliore amico, Luca, mi dice sempre che il muro rosso ha un potere speciale, che può far apparire fantasmi e creati magiche avventure. Eppure, nonostante la mia incredulità, una notte strana e oscura, qualcosa di inaspettato successe mentre stavo giocando sulla spiaggia...
Vivo a New York e l'amò. Vivo in un bell' appartamento nel quartiere di Manhattan, vicino al Central Park, il mio luogo preferito per passeggiare con il cane. Il rumore della città è sempre presente, ma non mi disturbano perché c'è sempre qualcosa da fare o vedere. Inoltre, ci sono i musei, il Metropolitan Museum of Art in particolare, che è il mio punto di riferimento per capire la cultura e l'arte del mondo. E poi, naturalmente, ci sono gli amici e la vita notturna della città, che non manca mai di sorprese e avventure.
Il giapponese